# Quantitative Bargain Stock Strategy


In [2]:
import numpy as np 
import pandas as pd 
import requests
import xlsxwriter 
import math
from scipy import stats

In [6]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()


In [ ]:
pe_ratio = data['peRatio']
pe_ratio


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)

#Fill out dataframe
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,99.160,46.39,N/A
1,AAL,13.430,-1.65,N/A
2,AAP,156.890,28.9,N/A
3,AAPL,475.040,34.8,N/A
4,ABBV,95.770,20.14,N/A
...,...,...,...,...
500,YUM,93.960,28.5,N/A
501,ZBH,143.010,723.05,N/A
502,ZBRA,299.348,31.74,N/A
503,ZION,34.700,13.44,N/A


In [ ]:
# Remove stocks that are not within top 50 in price to earnings ratio
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,SBAC,317.680,-2261.88,N/A
1,CRM,202.060,-1165.07,N/A
2,BMY,65.210,-639.08,N/A
3,TEL,99.630,-328.9,N/A
4,GILD,70.140,-318.24,N/A
5,HPE,9.720,-299.36,N/A
6,DRI,88.750,-247.4,N/A
7,KHC,36.964,-237.94,N/A
8,DIS,136.510,-221.5,N/A
9,XRAY,45.210,-203.96,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [ ]:
#Function for determining the ideal shares toi buy based on budget of portfolio
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()

In [ ]:
#Calculate the number of shares
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,SBAC,317.680,-2261.88,61
1,CRM,202.060,-1165.07,97
2,BMY,65.210,-639.08,300
3,TEL,99.630,-328.9,196
4,GILD,70.140,-318.24,279
5,HPE,9.720,-299.36,2017
6,DRI,88.750,-247.4,220
7,KHC,36.964,-237.94,530
8,DIS,136.510,-221.5,143
9,XRAY,45.210,-203.96,433


In [ ]:

#Using and displaying a better valuing strategy
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [ ]:
#Robust value columns
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

print("\n\n\n")

rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,100.010,N/A,46.95,N/A,6.596140,N/A,26.372976,N/A,11.311629,N/A,N/A
1,AAL,13.360,N/A,-1.71,N/A,-60.417952,N/A,5.952664,N/A,3.098483,N/A,N/A
2,AAP,161.440,N/A,29,N/A,3.123759,N/A,15.086368,N/A,3.208667,N/A,N/A
3,AAPL,466.070,N/A,34.69,N/A,22.373999,N/A,25.708773,N/A,19.428993,N/A,N/A
4,ABBV,96.680,N/A,21,N/A,-21.463532,N/A,12.272585,N/A,7.672427,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.320,N/A,28,N/A,-3.659682,N/A,18.841249,N/A,13.891510,N/A,N/A
501,ZBH,143.470,N/A,718.1,N/A,2.390128,N/A,17.170711,N/A,7.478460,N/A,N/A
502,ZBRA,288.222,N/A,31.86,N/A,8.600669,N/A,19.480804,N/A,8.449885,N/A,N/A
503,ZION,35.770,N/A,13.24,N/A,0.766237,N/A,NaN,N/A,NaN,N/A,N/A


In [ ]:
#If there is missing data, replace with "N/A"
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

rv_dataframe[rv_dataframe.isnull().any(axis=1)]

In [ ]:
#Calculate ratios
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.841584
1      0.112871
2      0.623762
3      0.740594
4      0.427723
         ...   
500         0.6
501    0.994059
502    0.693069
503    0.257426
504    0.843564
Name: PE Percentile, Length: 505, dtype: object
0       0.752475
1      0.0158416
2       0.510891
3       0.940594
4      0.0257426
         ...    
500     0.049505
501     0.415842
502     0.811881
503     0.132673
504     0.956436
Name: PB Percentile, Length: 505, dtype: object
0       0.877228
1      0.0732673
2        0.50099
3       0.861386
4       0.350495
         ...    
500     0.744554
501     0.572277
502     0.762376
503      0.69505
504     0.924752
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.552475
1      0.0574257
2      0.0653465
3       0.865347
4       0.340594
         ...    
500     0.744554
501     0.326733
502          0.4
503     0.644554
504     0.869307
Name: EV/GP Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,100.010,N/A,46.95,0.841584,6.596140,0.752475,26.372976,0.877228,11.311629,0.552475,N/A
1,AAL,13.360,N/A,-1.71,0.112871,-60.417952,0.0158416,5.952664,0.0732673,3.098483,0.0574257,N/A
2,AAP,161.440,N/A,29.00,0.623762,3.123759,0.510891,15.086368,0.50099,3.208667,0.0653465,N/A
3,AAPL,466.070,N/A,34.69,0.740594,22.373999,0.940594,25.708773,0.861386,19.428993,0.865347,N/A
4,ABBV,96.680,N/A,21.00,0.427723,-21.463532,0.0257426,12.272585,0.350495,7.672427,0.340594,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.320,N/A,28.00,0.6,-3.659682,0.049505,18.841249,0.744554,13.891510,0.744554,N/A
501,ZBH,143.470,N/A,718.10,0.994059,2.390128,0.415842,17.170711,0.572277,7.478460,0.326733,N/A
502,ZBRA,288.222,N/A,31.86,0.693069,8.600669,0.811881,19.480804,0.762376,8.449885,0.4,N/A
503,ZION,35.770,N/A,13.24,0.257426,0.766237,0.132673,18.729176,0.69505,12.206556,0.644554,N/A


In [ ]:
from statistics import mean


#Calculate the robust value score
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,100.010,N/A,46.95,0.841584,6.596140,0.752475,26.372976,0.877228,11.311629,0.552475,0.755941
1,AAL,13.360,N/A,-1.71,0.112871,-60.417952,0.0158416,5.952664,0.0732673,3.098483,0.0574257,0.0648515
2,AAP,161.440,N/A,29.00,0.623762,3.123759,0.510891,15.086368,0.50099,3.208667,0.0653465,0.425248
3,AAPL,466.070,N/A,34.69,0.740594,22.373999,0.940594,25.708773,0.861386,19.428993,0.865347,0.85198
4,ABBV,96.680,N/A,21.00,0.427723,-21.463532,0.0257426,12.272585,0.350495,7.672427,0.340594,0.286139
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.320,N/A,28.00,0.6,-3.659682,0.049505,18.841249,0.744554,13.891510,0.744554,0.534653
501,ZBH,143.470,N/A,718.10,0.994059,2.390128,0.415842,17.170711,0.572277,7.478460,0.326733,0.577228
502,ZBRA,288.222,N/A,31.86,0.693069,8.600669,0.811881,19.480804,0.762376,8.449885,0.4,0.666832
503,ZION,35.770,N/A,13.24,0.257426,0.766237,0.132673,18.729176,0.69505,12.206556,0.644554,0.432426


In [ ]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

#Calculate the number of shares to buy
portfolio_input()


position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

In [ ]:
#Put into an Excel
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [ ]:
#Add formating for Excel

background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [ ]:
writer.save()